In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

from scipy.stats import norm, t
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

# Pre-processing

## Import data

In [ ]:
df = pd.read_csv("../input/prediction-of-asteroid-diameter/Asteroid_Updated.csv", 
                 low_memory=False)

First look at the first 5 rows of data. Useful to get an initial feel for the dataset.

In [ ]:
pd.set_option('max_columns', 31)
df.head(5)

Check the shape and column names of the dataset.

In [ ]:
df.shape, df.columns

Look into the data types of each variable

In [ ]:
pd.DataFrame({"Column":df.columns, 
              "DType": df.dtypes.values, 
              "NonNulls": [df[col].count() for col in df.columns]})

## Cleaning

From above we can see our target feature, diameter, should be numeric but instead pandas has loaded it as an object dtype. 

It is likely there are some non-numerical values in the diameter column so we need to identify whether they can be converted and if not, either manually convert or remove them from the dataset.

In [ ]:
try: 
    df.diameter = pd.to_numeric(df.diameter)
except ValueError as e:
    print("ERROR:", e)

In [ ]:
converted_vals = []
for idx, val in enumerate(df.diameter.tolist()):
    if isinstance(val, str):
        converted_vals.append(float(val))
    else:
        converted_vals.append(float(val))
df.diameter = converted_vals
df.diameter = pd.to_numeric(df.diameter)
df.diameter.dtype

Drop features that have a proportion of null values greater than 5%.

In [ ]:
df_cleaned = df[df.diameter.notnull()]
df_cleaned = df_cleaned[[col for col in df_cleaned.columns if (df_cleaned[col].count() / len(df_cleaned)) > 0.95]]
df_cleaned.shape, df_cleaned.columns

In [ ]:
df_numerical = df_cleaned[[col for col, dtype in zip(df_cleaned.columns, df_cleaned.dtypes) if dtype != object]]
df_numerical.shape, df_numerical.columns

Drop rows that have values below 0

In [ ]:
def drop_negative(df):
    for col in df:
        if df[col].dtype == 'object':
            continue
        else:
            if df[col][df[col] < 0].count() > 0:
                df.drop(df[df[col] < 0].index, axis=0, inplace=True)
            else:
                continue
drop_negative(df_numerical)

Drop rows with one null

In [ ]:
df_numerical = df_numerical.dropna()
df_numerical.shape

In [ ]:
df_numerical.describe()

## Correlation Heatmap

In [ ]:
plt.figure(figsize=(16, 10))
ax = sns.heatmap(df_numerical[1:].corr(method='pearson'), 
            annot=True, 
            cmap=sns.color_palette('magma'),
            linewidth=0.3, 
            edgecolor='k')
bot, top = ax.get_ylim()        # To solve bug where sns.heatmap top and bottom axes
ax.set_ylim(bot + 0.5, top-0.5) # are cutoff in Matplotlib 3.1.1 (current version)
plt.title('Correlation Heatmap')
plt.show()

# EDA

In [ ]:
def VIF_calc(df):
    aux_df = df.assign(const=1)
    vif = pd.Series([variance_inflation_factor(aux_df.values, i) 
               for i in range(len(aux_df.columns))], 
              index=aux_df.columns)
    return vif

VIF_calc(df_numerical)

In [ ]:
df_numerical = df_numerical[[col for col, vif in zip(df_numerical.columns, VIF_calc(df_numerical).values) if vif < 5]]

#  Modeling

## Linear Regression (Base Model)

In [ ]:
def standard_error(df, col, rss_calc):
    x_bar = np.mean(df[col].tolist())
    feature_std = np.sqrt(sum([(xi - x_bar)**2 for xi in df[col].tolist()]))
    rse = np.sqrt(rss_calc / (len(df) - 2))
    return rse / feature_std

In [ ]:
def linear_model_eval(X, Y):
    
    reg = LinearRegression(normalize=False)

    kfold = KFold(n_splits = 10, shuffle=True, random_state = 42)

    scores = []
    rmses = []
    for train_index, test_index in kfold.split(X):
        x_train, x_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]

        reg.fit(x_train, y_train)
        scores.append([reg.score(x_test, y_test),
                       (np.sqrt(mean_squared_error(y_test, reg.predict(x_test))))])

    print("K-Fold CV:\n", pd.DataFrame(data = np.array(scores), columns = ['RSquared', 'RMSE']))

    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 999, shuffle = True)

    reg.fit(x_train, y_train)
    
    df_coef = pd.DataFrame(data = np.array([[m, n] for m, n in zip(X.columns,reg.coef_)]), columns = ['column', 'coef'])
    rss_calc = mean_squared_error(y_test, reg.predict(x_test)) * len(X)
    df_coef['SE'] = [standard_error(X, col, rss_calc) for col in X.columns]
    df_coef.coef = pd.to_numeric(df_coef.coef)
    df_coef['t'] = df_coef['coef'] / df_coef['SE']
    df_coef['pvalue'] = df_coef.apply(lambda row: t.sf(np.abs(row['t']), len(X)-1)*100 ,axis=1)
    print("Coefficient Analysis:\n", df_coef)
    
    print("R-Squared: {0:.3f} \nRMSE: {1:.3f}".format(reg.score(x_test, y_test), 
                                    np.sqrt(mean_squared_error(y_test, reg.predict(x_test)))))
    #print(reg.intercept_, np.sqrt(rss_calc*len(X)) * np.sqrt(1./len(X)) + )
    sns.distplot(y_test.values, kde=True, label = 'True')
    sns.distplot(reg.predict(x_test), kde=True, label = 'Prediction')
    plt.legend()
    plt.show()
    return reg

X = df_numerical.drop('diameter', axis=1)
Y = df_numerical.diameter

linear_model_eval(X,Y)

From the above analysis we can see that only **i**, **data_arc**, **albdeo**, and **n** have the greatest effect on the output of the model, statistically speaking. Therefore, removing all other variables should give us a very similar R-Squared.

In [ ]:
X = df_numerical[['i', 'data_arc', 'albedo', 'n']]
Y = df_numerical.diameter

linear_model_eval(X,Y)

In [ ]:
VIF_calc(X)

R-Squared is exactly the same with RMSE being very similar with just these 4 features, however it doesn't explain the variance very well as eluded to by a low R-Squared. Therefore, this first pass model doesn't include the correct features to properly explain the variance seen in the diameter.

## Reconstitution of features

Now that we have a base model, let's do some feature engineering to include features that we arbitrarily removed due to high VIF and try to glean a better fit. 

In [ ]:
print("Before:", df_cleaned.shape)
df_numerical = df_cleaned[[col for col, dtype in zip(df_cleaned.columns, df_cleaned.dtypes) if dtype != object]]
drop_negative(df_numerical)
df_numerical = df_numerical.dropna()
print("After:", df_numerical.shape)

Using the correlation heatmap, VIF scores, and the descriptions of the features we can drop:
 
* **per_y** - Orbital period in years : as this is given in days in **per**
* **moid** - Minimum Orbit Intersection Distance : as this is strongly correlated with the perihelion, **q**.

In [ ]:
df_numerical = df_numerical.drop(['per_y', 'moid'], axis=1)

### Linear model

In [ ]:
X = df_numerical.drop('diameter', axis=1)
Y = df_numerical.diameter

linear_model_eval(X,Y)

In [ ]:
VIF_calc(X)

Higher R-Squared achieved! Although we see a couple of features are insignificant and a few features have infinite VIF. Therefore these ought to be removed.

### Linear model -  $ln(diameter)$

Transform each column (features and target) by evaluating the natural log of each data point. Thanks to Liam Toran in his [notebook](https://www.kaggle.com/liamkesatoran/asteroid-diameter-estimators-with-added-difficulty) for suggesting a natural log transformation of diameter. 

Thus this generates a more robust, accurate, and precise linear model, as evidenced by the low standard errors and p-values below.

In [ ]:
X = df_numerical.drop(['diameter'], axis=1)
for col in X.columns:
    X["log "+ col] = X[col].apply(np.log)
    X = X.drop(col, axis=1)
#X = pd.DataFrame(StandardScaler().fit_transform(X), columns = X.columns)
Y = np.log(df_numerical.diameter)

linear_model_eval(X,Y)

The high R-Squared is suspicious however, we ought to check further for multicollinearity between features.

In [ ]:
VIF_calc(X)

As evidenced above, a number of features have a VIF > 5 with a few features that are directly proportional to other features.

Therefore, remove **per**, the Orbital Period in days, as this is an input into calculating **n**, the Mean motion, a parameter in Orbital Mechanics evaluated through the following equation:

$
\begin{equation}
\Large
n = \frac{2 \pi}{P}
\end{equation}
$

Also drop **a**, the semi-major axis, as it interacts with **q**, the perihelion distance, where **a** is the average between the perihelion, **q**, and aphelion, **ad**.

$
\large
ad = a(1-e)
\\
\large
q = a(1+e)
\\
\large
\therefore a = \frac{q + ad}{2}
$

In [ ]:
X = df_numerical.drop(['diameter', 'per', 'a', 'H', 'albedo'], axis=1)
for col in X.columns:
    X["log "+ col] = X[col].apply(np.log)
    X = X.drop(col, axis=1)
#X = pd.DataFrame(StandardScaler().fit_transform(X), columns = X.columns)
Y = np.log(df_numerical.diameter)

linear_model_eval(X,Y)

In [ ]:
VIF_calc(X)

There is still a high correlation between **ln(q)**, **ln(ad)** and **ln(n)**.

From knowledge of [Kepler's 3rd Law](https://en.wikipedia.org/wiki/Mean_motion#Mean_motion_and_Kepler's_laws), we understand that the semi-major axis (**a**) and mean motion (**n**) are proportional to one another through the relation:

$
\mu = a^3 n^2
$

where $\mu$ is the standard gravitational parameter in au/days for a heliocentric 2-body system.

Therefore, we can drop **q** and **ad** as they make up **a** and so are proportional to the Mean Motion **n**.

### Best non-log linear fit (standardised - no vif change)

In [ ]:
X = df_numerical.drop(['diameter', 'a', 'q', 'ad', 'per'], axis=1)
#X['n_obs_used:data_arc'] = X.n_obs_used * X.data_arc
#X['e:n'] = X.e * X.n
# X['H:n_obs_used'] = X.H * X.n_obs_used
X = pd.DataFrame(StandardScaler().fit_transform(X), columns = X.columns)
Y = np.log(df_numerical.diameter)

linear_model_eval(X,Y)

In [ ]:
VIF_calc(X)

### Best log linear fit

In [ ]:
X = df_numerical.drop(['diameter', 'a', 'q', 'ad', 'per'], axis=1)
X['e:n'] = X.e * X.n
for col in X.columns:
    X["log "+ col] = X[col].apply(np.log)
#     X = X.drop(col, axis=1)
X['log e:log n'] = X['log e'] * X['log n']
#X['log H: log n_obs_used'] = X['log H'] * X['log n_obs_used']
X = X.drop('log e:n', axis=1)
X = pd.DataFrame(StandardScaler().fit_transform(X), columns = X.columns)
Y = np.log(df_numerical.diameter)

linear_model_eval(X,Y)

In [ ]:
VIF_calc(X)

And just to prove that using **n** alone is better than using linear combinations of **q**, **ad** and **a**:

In [ ]:
X = df_numerical.drop(['diameter', 'n', 'per', 'albedo', 'H'], axis=1)
Y = np.log(df_numerical.diameter)

linear_model_eval(X,Y)

In [ ]:
VIF_calc(X)

In [ ]:
df_logged = df_numerical.copy()
# for col in df_logged.columns:
#     df_logged["ln "+ col] = df_logged[col].apply(np.log)
#     df_logged = df_logged.drop(col, axis=1)  
df_logged['ln diameter'] = np.log(df_logged.diameter)
for col in df_numerical.columns:
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 2, 1)
    sns.scatterplot(x=col, y='diameter', data=df_numerical)
    plt.title("Raw", fontsize=14)
    plt.subplot(1, 2, 2)
    sns.scatterplot(x=col, y='ln diameter', data=df_logged)
    plt.title("Logged", fontsize=14)
    plt.show()

## Gradient Boosting

### Base Model

In [ ]:
def ensemble_model_eval(X, Y):
    
    reg = GradientBoostingRegressor(
    loss='ls',
    learning_rate=0.1,
    n_estimators=1000,
    subsample=1.0,
    criterion='friedman_mse',
    min_samples_split=20,
    min_samples_leaf=20,
    min_weight_fraction_leaf=0.0,
    max_depth=10,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    init=None,
    random_state=42,
    max_features=None,
    alpha=0.9,
    verbose=0,
    max_leaf_nodes=50,
    warm_start=False,
    presort='auto',
    validation_fraction=0.1,
    n_iter_no_change=None,
    tol=0.0001)

#     kfold = KFold(n_splits = 10, shuffle=True, random_state = 42)

#     scores = []
#     for train_index, test_index in kfold.split(X):
#         x_train, x_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]

#         reg.fit(x_train, y_train)
#         scores.append([reg.score(x_test, y_test),
#                        (np.sqrt(mean_squared_error(y_test, reg.predict(x_test))))])

#     print("K-Fold CV:\n", pd.DataFrame(data = np.array(scores), columns = ['RSquared', 'RMSE']))

    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 42, shuffle = True)

    reg.fit(x_train, y_train)
    
    print("R-Squared: {0:.3f} \nRMSE: {1:.3f}".format(reg.score(x_test, y_test), 
                                    np.sqrt(mean_squared_error(y_test, reg.predict(x_test)))))
    
    sns.distplot(y_test.values, kde=True, label = 'True')
    sns.distplot(reg.predict(x_test), kde=True, label = 'Prediction')
    plt.legend()
    plt.show()
    return reg

X = df_numerical.drop(['diameter', 'albedo', 'H'], axis=1)
X['e:n'] = X.e * X.n
Y = np.log(df_numerical.diameter)

reg = ensemble_model_eval(X,Y)

In [ ]:
df_importance = pd.DataFrame(reg.feature_importances_, index = X.columns, columns=['Importance'])
df_importance.sort_values(by=['Importance'],  axis=0, ascending=False)

In [ ]:
def check_for_duplicates(df):
    return len(df.groupby(df.columns.tolist(), axis=0).size().reset_index().rename(columns={0:'count'})) == len(df)

check_for_duplicates(df_numerical)

In [ ]:
plt.figure(figsize=(15, 8))
plt.subplot(1, 2, 1)
plt.scatter(X.n, reg.predict(X), label='prediction')
plt.scatter(X.n, Y, label='true')
plt.legend()
plt.subplot(1, 2, 2)
plt.scatter(X.n, reg.predict(X) - Y, label='residuals')
plt.legend()
plt.show()

In [ ]:
plt.hist(reg.predict(X)-Y, bins=np.linspace(-3, 3, 201))
plt.show()